In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import gc
#!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev --version nightly
#20200420
#!pip install transformers==2.5.1
!pip install transformers==2.9.0
gc.collect()

In [ ]:
!pip install pandarallel > /dev/null

In [ ]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler

import time
import random
from datetime import datetime
from tqdm import tqdm
tqdm.pandas()

import re

# !pip install nltk > /dev/null
import nltk
nltk.download('punkt')

from nltk import sent_tokenize

from pandarallel import pandarallel

pandarallel.initialize(nb_workers=2, progress_bar=True)

In [ ]:
import os
import torch
import pandas as pd
from scipy import stats
import numpy as np

from tqdm import tqdm
from collections import OrderedDict, namedtuple
import torch.nn as nn
from torch.optim import lr_scheduler
import joblib
import re

import logging
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup, get_constant_schedule
import sys
from sklearn import metrics, model_selection

import warnings

warnings.filterwarnings("ignore")

#SEED=528491
SEED=42
MAX_LENGTH=224

In [ ]:
tokenizer = transformers.XLMRobertaTokenizer.from_pretrained('xlm-roberta-large')

In [ ]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_masks=True, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids']), np.array(enc_di['attention_mask'])

LANGS = {
    'en': 'english',
    'it': 'italian', 
    'fr': 'french', 
    'es': 'spanish',
    'tr': 'turkish', 
    'ru': 'russian',
    'pt': 'portuguese'
}

def get_sentences(text, lang='en'):
    return sent_tokenize(text, LANGS.get(lang, 'english'))

def exclude_duplicate_sentences(text, lang='en'):
    sentences = []
    for sentence in get_sentences(text, lang):
        sentence = sentence.strip()
        if sentence not in sentences:
            sentences.append(sentence)
        #else :
        #    print("-------------------------------------------")
        #    print(sentence)
        #    print("-------------------------------------------")
    return ' '.join(sentences)

def clean_text(text, lang='en'):
    text = str(text)
    text = re.sub(r'[0-9"]', '', text)
    text = re.sub(r'#[\S]+\b', '', text)
    text = re.sub(r'@[\S]+\b', '', text)
    text = re.sub(r'https?\S+', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = exclude_duplicate_sentences(text, lang)
    return text.strip()

def clean_text(text, lang='en'):
    text = str(text)
    text = re.sub(r'[0-9"]', '', text)
    text = re.sub(r'#[\S]+\b', '', text)
    text = re.sub(r'@[\S]+\b', '', text)
    text = re.sub(r'https?\S+', '', text)
    text = re.sub(r'\s+', ' ', text)
    #text = exclude_duplicate_sentences(text, lang)
    return text.strip()

In [ ]:
langs=['tr','it','es','ru','fr','pt']
#langs=['ru','fr','pt']
#langs=['tr','it','es']
D_TRANS='/kaggle/input/jigsaw-train-multilingual-coments-google-api/'

for i in range(len(langs)):
    print(langs[i])
    fn = D_TRANS+'jigsaw-toxic-comment-train-google-%s-cleaned.csv'%langs[i]
    
    df_train = pd.read_csv(fn, usecols=["comment_text", "toxic"]).fillna("none")
    n_samples_1 = df_train[df_train['toxic'] == 1].shape[0]
    
    # Combine train1 with a subset of train2
    df_train = pd.concat([
        df_train[['comment_text', 'toxic']].query('toxic==1'),
        df_train[['comment_text', 'toxic']].query('toxic==0').sample(n=n_samples_1, random_state=SEED)
        ])

    df_train = df_train.sample(frac=1).reset_index(drop=True)
    print(df_train.shape)
    print("-------------------------")
    print("n_samples_1 : {}".format(n_samples_1))
    
    df_train["comment_text"] = df_train.parallel_apply(lambda x: clean_text(x['comment_text']), axis=1)
    x_train_id, x_train_mask = regular_encode(df_train.comment_text.values, tokenizer, maxlen=MAX_LENGTH)
    y_train                  = df_train.toxic.values
    
    np.save('x_train_id_%s'%langs[i], x_train_id)
    np.save('x_train_mask_%s'%langs[i], x_train_mask)
    np.save('y_train_%s'%langs[i], y_train)
    
    del(df_train)
    del(x_train_id)
    del(x_train_mask)
    del(y_train)
    gc.collect()